In [5]:
import pandas as pd 
import wbdata

## Access

In [11]:
countries = {"Burkina Faso":"BF", "Congo, Dem. Rep.":"CD", "Ethiopia":"ET", "Kenya":"KE", "Nigeria":"NG", 
          "Senegal":"SN", "Tanzania":"TZ", "Uganda":"UG", "South Africa":"ZA", "Zambia":"ZM"}

country_code = list({v for (k,v) in countries.items()})

In [13]:
def collect():
    # generate a dict from the indicators file
    takwimu_indicators = pd.read_csv('Takwimu_indicators.csv',
                                     index_col=0, squeeze=True).to_dict()
    # Gather indicator data on the selected countries
    wb_data = wbdata.get_dataframe(takwimu_indicators, 
                                 country=country_code, convert_date=False)
    return wb_data    

## Structure

In [55]:
def process():
    df = pd.read_csv('data/takwimu_wb_data.csv')
    
#     population
    def population():
        df_pop = data[['country', 'date','Population Male', 'PopulationFemale' ]]
        df_pop = df_pop[df_pop['date']==2017].drop('date', axis=1)
        df_pop.columns = ['name','male','female']


        df_pop = df_pop.melt(id_vars='name', value_vars=['male','female'], 
                var_name='sex', value_name='total')

        df_pop['geo_code'] = df_pop['name'].map(countries)
        df_pop['geo_level'] = "country"
        df_pop['geo_version'] = 2017

        population = df_pop[['geo_level','geo_code','name','geo_version','sex','total']].sort_values('name')

        return population
    
#     basic services
    basic_services = data[['country', 'date','access to basic services - Electricity','access to basic services - Water' ]]
    basic_services = basic_services[basic_services['date']==2015]
    basic_services.columns = ['name','geo_version','electricity','water']

    return population, basic_services
    

In [91]:
def population():
    df_pop = data[['country', 'date','Population Male', 'PopulationFemale' ]]
    df_pop = df_pop[df_pop['date']==2017].drop('date', axis=1)
    df_pop.columns = ['name','male','female']


    df_pop = df_pop.melt(id_vars='name', value_vars=['male','female'], 
            var_name='sex', value_name='total')

    df_pop['geo_code'] = df_pop['name'].map(countries)
    df_pop['geo_level'] = "country"
    df_pop['geo_version'] = 2017

    population = df_pop[['geo_level','geo_code','name','geo_version','sex','total']].sort_values('name')

    return population
    


pandas.core.frame.DataFrame

In [74]:
df_pop = data[['country', 'date','Population Male', 'PopulationFemale' ]]
df_pop = df_pop[df_pop['date']==2017].drop('date', axis=1)
df_pop.columns = ['name','male','female']

df_pop

,name,male,female
0,Burkina Faso,9571934.0,9621448.0
58,"Congo, Dem. Rep.",40581481.0,40758507.0
116,Ethiopia,52406026.0,52551412.0
174,Kenya,24700721.0,24999141.0
232,Nigeria,96729234.0,94157077.0
290,Senegal,7787318.0,8063249.0
348,Tanzania,28341970.0,28968049.0
406,Uganda,21320842.0,21542116.0
464,South Africa,27835727.0,28881429.0
522,Zambia,8482226.0,8611904.0


In [24]:
data = pd.read_csv('data/takwimu_wb_data.csv')

data.head()

,country,date,Population Male,PopulationFemale,access to basic services - Electricity,access to basic services - Water,Youth unemployment-Male,Youth unemployment - Female,Life expectancy-Male,Life expectancy-Female,...,Secondary school enrolment - Male (% gross),Secondary school enrolment - Female (% gross),Literacy rate - Male,Literacy rate - Female,Debt servicing % of GNI,FDI,"Inflation, consumer prices (annual %)",Non performing loans,health expenditure (% of GDP),Expenditure of Education (% GDP)
0,Burkina Faso,2017,9571934.0,9621448.0,NaN,NaN,5.471,12.767,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.368047,NaN,NaN,NaN
1,Burkina Faso,2016,9292562.0,9353871.0,19.164713,NaN,5.424,12.801,59.598,60.992,...,36.668530,34.871910,NaN,NaN,1.091555,-3.398425e+08,-0.244764,NaN,NaN,NaN
2,Burkina Faso,2015,9018776.0,9091848.0,18.470081,53.857596,5.392,12.861,59.191,60.531,...,35.066872,32.320610,NaN,NaN,1.123317,-2.176320e+08,0.954993,NaN,5.446772,4.16607
3,Burkina Faso,2014,8750599.0,8835378.0,19.200000,53.341326,5.392,12.863,58.747,60.035,...,32.368580,28.267019,44.379452,26.220249,0.775892,-2.873679e+08,-0.258090,NaN,5.554305,4.55994
4,Burkina Faso,2013,8488178.0,8584545.0,17.084827,52.826072,5.235,11.756,58.255,59.496,...,30.680510,25.969860,NaN,NaN,0.672815,-4.319736e+08,0.533739,NaN,6.059873,4.59384


In [84]:
data.columns

Index(['country', 'date', 'Population Male', 'PopulationFemale',
       'access to basic services - Electricity',
       'access to basic services - Water', 'Youth unemployment-Male',
       'Youth unemployment - Female', 'Life expectancy-Male',
       'Life expectancy-Female', 'GDP-Current US $', 'GDP Growth',
       'GDP Per Capita', 'GDP per capita (annual %)',
       'Tax revenue (current LCU)', 'Tax revenues', 'GINI Coefficent',
       'Poverty headcount ratio at $1.90 (% of population)',
       'Prevalence of undernourishment',
       'Births attended by skilled health staff', 'Infant Mortality',
       'Under 5 Mortality rates', 'Prevalence of HIV, male (% ages 15-24)',
       'Prevalence of HIV, female (% ages 15-24)',
       'Primary completion rate, male (%)',
       'Primary completion rate, female (%)', 'female genital mutilation',
       'Labor force, female (% of total labor force)',
       'Employment to population ratio male (%)',
       'Employment to population ratio 